In [2]:
import pandas as pd
import pymysql
import json
import warnings
import DB
from collections import defaultdict
from collections import Counter
from pprint import pprint
warnings.filterwarnings(action='ignore')

In [77]:

db = DB.MYDB()
keyplayers_total = dict()
for s in range(15, 40):
    keyplayers_count = defaultdict(list)
    db.cursor.execute("select * from keyplayer where seasonCode = {}".format(s))
    for keyplayer in db.cursor.fetchall():
        pcode = keyplayer['pcode']
        tcode = keyplayer['tcode']
        keyplayers_count[tcode].append(pcode)
    for k, v in keyplayers_count.items():
        keyplayers_count[k] = Counter(v)
    keyplayers_total[s] = keyplayers_count
db.db.close()
    


Connect!


In [20]:
item = pd.read_csv('recent_avg_record_ver2_10game.csv')
result = pd.DataFrame()
item.columns

Index(['Unnamed: 0', 'gmkey', 'tcode', 'score', 'loss', 'benchScoreCn', 'bf',
       'bs', 'defr', 'dk', 'dkA', 'ef', 'fb', 'fbScoreCn', 'fg', 'fgA', 'fgt',
       'fgtA', 'foul', 'foulout', 'ft', 'ftA', 'gd', 'idf', 'inout', 'inout1',
       'maxContiScoreCn', 'maxLeadScoreCn', 'offr', 'playMin', 'playSec', 'pp',
       'ppA', 'pscore', 'rb', 'scoreHighTimeCn', 'secChanceScoreCn', 'ast',
       'teamR', 'tf', 'tfb', 'threep', 'threepA', 'to', 'tto',
       'turnoverScoreCn', 'wft', 'woft', 'stl'],
      dtype='object')

In [21]:
#게임스코어
#득점 + 0.4*야투성공개수 - 0.7*야투시도갯수 - 0.4*(자유투시도갯수 - 자유투성공갯수) + 
#0.7 * 오펜스리바운드 + 0.3 * 디펜스리바운드 + 스틸 + 0.7 * 어시스트 + 0.7 * 블럭 - 0.4 * 개인파울 - 턴오버
result['gameScore'] = item['score'] + 0.4 * item['fgt'] - 0.7 * item['fgtA'] - 0.4*(item['ftA'] - item['ft']) + \
0.7 * item['offr'] + 0.3 * item['defr'] + item['stl'] + 0.7 * item['ast'] + 0.7 * item['bs'] - 0.4 * item['foul'] - item['to']

#승률
result['winRate'] = -3.590 + 4.112 * item['defr'] + 1.839 * item['fg'] / item['fgA'] + 4.474 * item['stl'] + \
    1.674 * item['threep'] / item['threepA'] - 2.2 * item['to'] + 1.109 * item['ftA'] + 2.320 * item['bs'] + \
    1.812 * item['offr'] + 0.650 * item['ft'] / item['ftA'] + 0.590 * item['ast']

#팀포제션
result['teamPossesion'] = item['fgtA'] + 0.44 * item['ftA'] - item['offr'] + item['tto']

#PMG
result['PMG'] = result['teamPossesion'] - item['fg'] - 0.77*(item['fgA'] - item['fg']) - 0.44 * item['ft'] - \
    0.339 * (item['ftA'] - item['ft']) - item['to'] + 0.77 * item['offr'] + 0.55 * item['ast'] + \
    item['stl'] + 0.23 * item['defr'] * 0.2 * item['foul'] + 0.7 * item['bs']


result['win'] = item['score'] > item['loss']

In [23]:
result[:, :-1]

InvalidIndexError: (slice(None, None, None), slice(None, -1, None))